Setup:

1) Make sure GPU is enabled, go to edit->notebook settings->Hardware Accelerator GPU

2) Make a copy to your google drive, click on copy to drive in panel

Note: Colab will reset after 12 hours make sure to save your model checkpoints to google drive around 10-11 hours mark or before, then go to runtime->reset all runtimes. Now copy your train model back into colab and start training again from the previous checkpoint.

#Installing requirements & preparing training

clone and cd into repo, nshepperd's fork https://github.com/nshepperd/gpt-2

In [ ]:
!git clone https://github.com/nshepperd/gpt-2.git

Cloning into 'gpt-2'...
remote: Enumerating objects: 366, done.
remote: Total 366 (delta 0), reused 0 (delta 0), pack-reused 366
Receiving objects: 100% (366/366), 4.42 MiB | 2.94 MiB/s, done.
Resolving deltas: 100% (198/198), done.


In [ ]:
cd gpt-2

/content/gpt-2


Install requirements

In [ ]:
from datetime import datetime
from google.colab import files

MARLOES edit: try to install the correct tensorflow version (if this does not work, first uninstall tensorflow)

In [ ]:
!pip install tensorflow==1.15
!pip install tensorflow-gpu==1.15
!pip install 'tensorflow-estimator<1.15.0rc0,>=1.14.0rc0' --force-reinstall

     |████████████████████████████████| 412.3MB 39kB/s 
     |████████████████████████████████| 512kB 65.7MB/s 
     |████████████████████████████████| 3.8MB 55.3MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=93fa9f1e3777370abbea5fb313a4a303e9955a09d5ede8057adc71994b93dc6b
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
  Found existing installation: tensorflow-estimator 2.2.0rc0
    Uninstalling tensorflow-estimator-2.2.0rc0:
      Successfully uninstalled tensorflow-estimator-2.2.0rc0
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorboard 2.1.1
    Uninstalling tensorboard-2.1.1:
      Successfully uninstalled tensorboard-2.1.1
  Found existing installation: tensorflow 2.2.0rc1
    Uninstalling tensorflow-2.2.0rc1:
      Successfully uninstalled tensorflow-2.2.0rc

In [ ]:
!pip3 install -r requirements.txt

     |████████████████████████████████| 81kB 1.5MB/s 
     |████████████████████████████████| 604kB 4.1MB/s 
     |████████████████████████████████| 51kB 7.8MB/s 
  Created wheel for fire: filename=fire-0.3.0-py2.py3-none-any.whl size=111108 sha256=dad9ed5aacdeb80dfb12f7bfd7bae2deab9368285150b487d7dfbb8530f06bf6
  Stored in directory: /root/.cache/pip/wheels/92/62/79/6a40acd827ec9d78d610be311820ecf8e41db024d8b1d12ace
  Created wheel for regex: filename=regex-2017.4.5-cp36-cp36m-linux_x86_64.whl size=533194 sha256=ea5df0863d17b2f963a51e32fe042d68dde562f285110b88a7779eeef0ebfbb3
  Stored in directory: /root/.cache/pip/wheels/75/07/38/3c16b529d50cb4e0cd3dbc7b75cece8a09c132692c74450b01
Successfully built fire regex
ERROR: spacy 2.2.4 has requirement tqdm<5.0.0,>=4.38.0, but you'll have tqdm 4.31.1 which is incompatible.
  Found existing installation: regex 2019.12.20
    Uninstalling regex-2019.12.20:
      Successfully uninstalled regex-2019.12.20
  Found existing installation: tqdm 4.38.

Mount drive to access google drive for saving and accessing checkpoints later. Have to log in to your google account

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Download the model data

In [ ]:
!python3 download_model.py 345M

Fetching checkpoint: 1.00kit [00:00, 774kit/s]                                                      
Fetching encoder.json: 1.04Mit [00:00, 52.0Mit/s]                                                   
Fetching hparams.json: 1.00kit [00:00, 1.07Mit/s]                                                   
Fetching model.ckpt.data-00000-of-00001: 1.42Git [00:18, 77.1Mit/s]                                 
Fetching model.ckpt.index: 11.0kit [00:00, 8.41Mit/s]                                               
Fetching model.ckpt.meta: 927kit [00:00, 64.0Mit/s]                                                 
Fetching vocab.bpe: 457kit [00:00, 63.7Mit/s]                                                       


encoding

In [ ]:
!export PYTHONIOENCODING=UTF-8

Fetch checkpoints if you have them saved in google drive

In [ ]:
!cp -r /content/drive/My\ Drive/checkpoint/ /content/gpt-2/ 

Upload training text file from computer or google drive to google colab space, and move to folder gpt-2.

#Charles Dickens text


Let's get our train on! In this case the file is A Tale of Two Cities (Charles Dickens) from Project Gutenberg. To change the dataset GPT-2 models will fine-tune on, change this URL to another .txt file, and change corresponding part of the next cell. Note that you can use small datasets if you want but you will have to be sure not to run the fine-tuning for too long or you will overfit badly. Roughly, expect interesting results within minutes to hours in the 1-10s of megabyte ballpark, and below this you may want to stop the run early as fine-tuning can be very fast.

In [ ]:
!wget https://www.gutenberg.org/files/98/98-0.txt

--2020-03-12 14:57:52--  https://www.gutenberg.org/files/98/98-0.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 804335 (785K) [text/plain]
Saving to: ‘98-0.txt’

98-0.txt            100%[===================>] 785.48K   521KB/s    in 1.5s    

2020-03-12 14:57:54 (521 KB/s) - ‘98-0.txt’ saved [804335/804335]



#Finetuning the model


Start training, add --model_name '345M' to use 345 model

**possible to change:**   
    
    '--restore_from', type=str, default='latest', help='Either "latest", "fresh", or a path to a checkpoint file'

    '--sample_every', metavar='N', type=int, default=100, help='Generate samples every N steps'

    '--model_name', metavar='MODEL', type=str, default='117M', help='Pretrained model name, change to 345M if you downloaded that one'

**standard settings:**
top_k=40
top_p=0.0


In [ ]:
!PYTHONPATH=src ./train.py --dataset /content/gpt-2/honesty.txt --model_name '345M' --sample_every 2000 --restore_from "fresh"


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.




2020-03-27 13:49:19.255115: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2020-03-27 13:49:19.269184: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2020-03-27 13:49:19.269807: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x1320f40 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-03-27 13:49:19.269879: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2020-03-27 13:49:19.313204

Save our checkpoints to start training again later

In [ ]:
!cp -r /content/gpt-2/checkpoint/ /content/drive/My\ Drive/

#Loading finetuned model



Load your trained model for use in sampling below (117M or 345M)

In [ ]:
#!cp -r /content/gpt-2/checkpoint/run1/* /content/gpt-2/models/117M/

In [ ]:
!cp -r /content/gpt-2/checkpoint/run1/* /content/gpt-2/models/345M/

Generate conditional samples from the model given a prompt you provide -  change top-k hyperparameter if desired (default is 40),  if you're using 345M, add "--model-name 345M"



# Conditional Sampling

In [ ]:
!python3 src/interactive_conditional_samples.py --top_k 40 --model_name "345M"

In [ ]:
!python3 src/interactive_conditional_samples.py --top_k 40 --temperature=1 --length=200 --model_name "345M"



2020-03-12 15:01:02.043769: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-03-12 15:01:02.083214: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-03-12 15:01:02.084391: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: Tesla K80 major: 3 minor: 7 memoryClockRate(GHz): 0.8235
pciBusID: 0000:00:04.0
2020-03-12 15:01:02.128060: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2020-03-12 15:01:02.415279: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2020-03-12 15:01:02.570313: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.so.10


To check flag descriptions, use:

In [ ]:
!python3 src/interactive_conditional_samples.py -- --help

# Unconditional Sample Generation code 

Generate unconditional samples from the model,  if you're using 345M, add "--model-name 345M"


In [ ]:
!python3 src/generate_unconditional_samples.py --nsamples=1 --model_name "345M" | tee /tmp/samples > H_generated_samples.txt



2020-03-29 15:30:14.448489: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-03-29 15:30:14.486461: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-03-29 15:30:14.487075: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: Tesla P100-PCIE-16GB major: 6 minor: 0 memoryClockRate(GHz): 1.3285
pciBusID: 0000:00:04.0
2020-03-29 15:30:14.496101: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.0
2020-03-29 15:30:14.707957: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10.0
2020-03-29 15:30:14.812480: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library li

In [ ]:
files.download('H_generated_samples.txt')  

In [ ]:
# don't use !python3 src/generate_unconditional_samples.py --nsamples=50 --top_k=40 --temperature=1 --model_name "345M" | tee /tmp/samples > DH_generated_samples_2.txt



2020-03-25 10:14:35.357692: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-03-25 10:14:35.392790: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-03-25 10:14:35.393317: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: Tesla T4 major: 7 minor: 5 memoryClockRate(GHz): 1.59
pciBusID: 0000:00:04.0
2020-03-25 10:14:35.393618: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.0
2020-03-25 10:14:35.394773: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10.0
2020-03-25 10:14:35.395882: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.so.10.0

In [ ]:
# files.download('DH_generated_samples_2.txt') 

In [ ]:
#gen_file = 'gpt2_gentext_{:%Y%m%d_%H%M%S}.txt'.format(datetime.utcnow()) #does not work
#!python3 src/generate_unconditional_samples.py --nsamples=25 --top_k=40 --model_name "345M" --destination_path=gen_file | tee /tmp/samples

To check flag descriptions, use:

In [ ]:
!python3 src/generate_unconditional_samples.py -- --help